In [13]:
import datetime
import numpy as np
import pandas as pd


# 遗留问题
# 1. 不同文件的日期格式不同
# 2. 热值换算，单位有多个，还没来得及看，当面讨论
# 3. 建议分工: 1人review；1人写批量处理代码；1人写热值换算代码


class CoalAnalysis:
    def __init__(self, record):
        # 能否换算
        self.convertible = True
          
        # 日期（不同文件的日期格式不同）
        self.date = datetime.datetime.strptime(str(record['入炉日期']), "%Y-%m-%d").strftime("%Y-%m-%d")
        
        # 入炉煤量
        self.coal_comsuption = record['入炉煤量（吨）']
        
        # 收到基: Mar + Var + FCar + Aar = 100
        self.Mar = record['全水分/收到基水分 Mt/Mar (%)']
        self.Var = record['收到基挥发分 Var(%)']
        self.FCar = record['收到基固定碳  FCar(%)']
        self.Aar = record['收到基灰分 Aar(%)']
        
        # 空气干燥基: Mad + Vad + FCad + Aad = 100
        self.Mad = record['空干基水分 Mad (%)']
        self.Vad = record['空干基挥发分 Vad (%)']
        self.FCad = record['空干基固定碳 FCad(%)']
        self.Aad = record['空干基灰分 Aad (%)']
        
        # 干燥基: Vd + FCd + Ad = 100
        self.Vd = np.nan
        self.FCd = np.nan
        self.Ad = record['干燥基灰分Ad(%)']
        
        # 干燥无灰基: Vdaf + FCdaf = 100
        self.Vdaf = record['干燥无灰基挥发分 Vdaf(%)']
        self.FCdaf = np.nan
        
        # 全硫
        self.Star = record['收到基全硫St,ar%']
        self.Stad = record['空干基全硫 St,ad(%)']
        self.Std = record['干燥基全硫St,d%']


    # 检查数据能否换算: 1) 必须包含 Mar 和 Mad; 2) Aar、Aad、Ad至少包含一个
    def is_convertible(self):
        if (np.isnan(self.Mar) == True or np.isnan(self.Mad) == True) \
            or (np.isnan(self.Aar) == True \
                and np.isnan(self.Aad) == True \
                and np.isnan(self.Ad) == True):
            self.convertible = False
        return self.convertible


    # 计算四种基的灰分
    def calculate_A(self):
        # 缺少收到基灰分
        if np.isnan(self.Aar):
            # 空干基灰分 --> 收到基灰分
            if not np.isnan(self.Aad):
                self.Aar = self.Aad * (100 - self.Mar) / (100 - self.Mad)
            # 干燥基灰分 --> 收到基灰分
            elif not np.isnan(self.Ad):
                self.Aar = self.Ad * (100 - self.Mar) / 100

        # 缺少空干基灰分
        if np.isnan(self.Aad):
            self.Aad = self.Aar * (100 - self.Mad) / (100 - self.Mar)

        # 缺少干燥基灰分
        if np.isnan(self.Ad):
            # 收到基灰分 --> 干燥基灰分
            self.Ad = self.Aar * 100 / (100 - self.Mar)


    # 计算四种基的挥发分
    def calculate_V(self):
        # 缺少收到基灰挥发分
        if np.isnan(self.Var):
            # 空干基挥发分 --> 收到基挥发分
            if not np.isnan(self.Vad):
                self.Var = self.Vad * (100 - self.Mar) / (100 - self.Mad)
            # 干燥基挥发分 --> 收到基挥发分
            elif not np.isnan(self.Vd):
                self.Var = self.Vad * (100 - self.Mar) / 100
            # 干燥无灰基挥发分 --> 收到基挥发分
            elif not np.isnan(self.Vdaf):
                self.Var = self.Vdaf * (100 - self.Mar - self.Aar) / 100
            # 100 - (Mar + Aar + FCar)
            else:
                if not (np.isnan(self.Aar) and np.isnan(self.FCar)):
                    self.Var = 100 - self.Mar - self.Aar - self.FCar
        
        # 缺少空干基挥发分
        if np.isnan(self.Vad):
            self.Vad = self.Var * (100 - self.Mad) / (100 - self.Mar)
        
        # 缺少干燥基挥发分
        if np.isnan(self.Vd):
            self.Vd = self.Var * 100 / (100 - self.Mar)
        
        # 缺少干燥无灰基挥发分
        if np.isnan(self.Vdaf):
            self.Vdaf = self.Var * 100 / (100 - self.Mar - self.Aar)


    # 计算四种基的固定碳
    def calculate_FC(self):
        # 缺少收到基灰固定碳
        if np.isnan(self.FCar):
            # 空干基固定碳 --> 收到基固定碳
            if not np.isnan(self.FCad):
                self.FCar = self.FCad * (100 - self.Mar) / (100 - self.Mad)
            # 干燥基固定碳 --> 收到基固定碳
            elif not np.isnan(self.FCd):
                self.FCar = self.FCad * (100 - self.Mar) / 100
            # 干燥无灰基固定碳 --> 收到基固定碳
            elif not np.isnan(self.FCdaf):
                self.FCar = self.FCdaf * (100 - self.Mar - self.Aar) / 100
            # 100 - (Mar + Var + Aar)
            else:
                if not (np.isnan(self.Var) and np.isnan(self.Aar)):
                    self.FCar = 100 - self.Mar - self.Var - self.Aar
        
        # 缺少空干基固定碳
        if np.isnan(self.FCad):
            self.FCad = self.FCar * (100 - self.Mad) / (100 - self.Mar)
        
        # 缺少干燥基固定碳
        if np.isnan(self.FCd):
            self.FCd = self.FCar * 100 / (100 - self.Mar)
        
        # 缺少干燥无灰基固定碳
        if np.isnan(self.FCdaf):
            self.FCdaf = self.FCar * 100 / (100 - self.Mar - self.Aar)
    
    
    # 计算四种基的全硫
    def calculate_St(self):
        # 缺少收到基灰全硫
        if np.isnan(self.Star):
            # 空干基全硫 --> 收到基全硫
            if not np.isnan(self.Stad):
                self.Star = self.Stad * (100 - self.Mar) / (100 - self.Mad)
            # 干燥基全硫 --> 收到基全硫
            elif not np.isnan(self.Std):
                self.Star = self.Stad * (100 - self.Mar) / 100
            # 干燥无灰基全硫 --> 收到基全硫
            elif not np.isnan(self.Stdaf):
                self.Star = self.Stdaf * (100 - self.Mar - self.Aar) / 100
        
        # 缺少空干基全硫
        if np.isnan(self.Stad):
            self.Stad = self.Star * (100 - self.Mad) / (100 - self.Mar)
        
        # 缺少干燥基全硫
        if np.isnan(self.Std):
            self.Std = self.Star * 100 / (100 - self.Mar)


# 测试
data = pd.read_excel("test.xlsx")
for i in range(len(data)):
    record = CoalAnalysis(data.iloc[i])
    if record.is_convertible():
        record.calculate_A()
        record.calculate_V()
        record.calculate_FC()
        record.calculate_St()
        print(record.__dict__)
        # break

{'convertible': True, 'date': '2020-01-01', 'coal_comsuption': 9800, 'Mar': 12.26, 'Var': 24.806407908428724, 'FCar': 44.05260145681582, 'Aar': 18.880990634755463, 'Mad': 3.9, 'Vad': 27.17, 'FCad': 48.25, 'Aad': 20.68, 'Vd': 28.27263267429761, 'FCd': 50.208116545265355, 'Ad': 21.519250780437, 'Vdaf': 36.0249270750464, 'FCdaf': 63.9750729249536, 'Star': 0.8308366285119668, 'Stad': 0.91, 'Std': 0.9469302809573362}
{'convertible': True, 'date': '2020-01-02', 'coal_comsuption': 10858, 'Mar': 12.75, 'Var': 25.962908633696564, 'FCar': 44.255788977367985, 'Aar': 17.03130238893546, 'Mad': 4.56, 'Vad': 28.4, 'FCad': 48.41, 'Aad': 18.63, 'Vd': 29.756915339480305, 'FCd': 50.72296730930428, 'Ad': 19.5201173512154, 'Vdaf': 36.9743522978779, 'FCdaf': 63.02564770212213, 'Star': 0.6764983235540654, 'Stad': 0.74, 'Std': 0.7753562447611063}
{'convertible': True, 'date': '2020-01-03', 'coal_comsuption': 11543, 'Mar': 12.25, 'Var': 26.64605934907467, 'FCar': 44.03366305041481, 'Aar': 17.07027760051053, 'M